In [1]:
import numpy as np
import pandas as pd
import os
import random
random.seed(1)

from tqdm import tqdm_notebook as tqdm

import lightgbm as lgb

In [2]:
input_path = '../datasets/'

In [3]:
log_df = pd.read_csv(os.path.join(input_path, 'carlog.csv'))
meta_df = pd.read_csv(os.path.join(input_path, 'meta.csv'))
test_df = pd.read_csv(os.path.join(input_path, 'test.csv'))
product_master_df = pd.read_csv(os.path.join(input_path, 'product_master.csv'))
user_master_df = pd.read_csv(os.path.join(input_path, 'user_master.csv'))
sample_submission_df = pd.read_csv(os.path.join(input_path, 'atmaCup#9__sample_submission.csv'))

/Users/imazekishouta/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# 予測部分の設定

## TrainデータとTestデータによって処理を分ける

In [50]:
def split_trian_test_log(df, test_df):
    train_log_df = df[~df['session_id'].isin(test_df['session_id'])]
    test_log_df = df[df['session_id'].isin(test_df['session_id'])]
    return train_log_df, test_log_df

train_log_df, test_log_df = split_trian_test_log(log_df, test_df)
display(train_log_df.head())
display(test_log_df.head())

,session_id,register_number,date,hour,spend_time,display_action_id,user_id,value_1,name_1,kind_1,number_1,value_2,name_2,kind_2,number_2,unit_price,n_items,is_payment,predict
0,0,2003,2019-02-14,4,0,136,bE94Gct4vGiKM7z2Y79cjk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,0,2003,2019-02-14,4,32,209,bE94Gct4vGiKM7z2Y79cjk,4522646719765,おかかおにぎり ５９円,商品,1.0,NaN,NaN,NaN,NaN,59.0,1,0,0
2,0,2003,2019-02-14,4,37,209,bE94Gct4vGiKM7z2Y79cjk,4522646695663,梅（種なし）おにぎり５９,商品,1.0,NaN,NaN,NaN,NaN,59.0,1,0,0
3,0,2003,2019-02-14,4,52,209,bE94Gct4vGiKM7z2Y79cjk,4522646716641,紅鮭おにぎり７９円,商品,1.0,NaN,NaN,NaN,NaN,79.0,1,0,0
4,0,2003,2019-02-14,4,59,209,bE94Gct4vGiKM7z2Y79cjk,4522646695670,明太子おにぎり ７９円,商品,1.0,NaN,NaN,NaN,NaN,79.0,1,0,0


,session_id,register_number,date,hour,spend_time,display_action_id,user_id,value_1,name_1,kind_1,number_1,value_2,name_2,kind_2,number_2,unit_price,n_items,is_payment,predict
14299716,663721,1018,2020-08-01,8,0,137,NaN,2960001023607,スキャンデータ,バーコードスキャン,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
14299717,663725,1082,2020-08-01,8,0,137,NaN,2960050321792,スキャンデータ,バーコードスキャン,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
14299718,663725,1082,2020-08-01,8,31,209,FDhroMKMFKGXcvCV9t5e7N,4902806212259,GBﾑｰﾋﾞﾝｸﾞﾛｯｸ,商品,1.0,NaN,NaN,NaN,NaN,599.0,1,0,0
14299719,663725,1082,2020-08-01,8,192,209,FDhroMKMFKGXcvCV9t5e7N,4970458710171,替えブラシ,商品,1.0,NaN,NaN,NaN,NaN,298.0,1,0,0
14299720,663725,1082,2020-08-01,9,433,209,FDhroMKMFKGXcvCV9t5e7N,4904230027621,ブラックニッカクリアブレ,商品,1.0,NaN,NaN,NaN,NaN,699.0,1,0,0


## Trainデータ: 乱数を使って、予測すべき部分を設定する

In [51]:
def split_train_predict(df):
    df['predict'] = 0 # 次、もし実行するならis_predictにする
    target_idxes = []
    session_ids = list(df['session_id'].unique())
    for session_id in tqdm(session_ids):
        session_df = df[df['session_id']==session_id] # 対象セッションに絞る
        target_spend_time = random.randint(0, 100) # spend_timeの閾値設定
        target_session_df = session_df[session_df['spend_time'] >= target_spend_time]
        target_idxes += list(target_session_df.index)
    df.loc[target_idxes, 'predict'] = 1
    return df

train_log_df2 = split_train_predict(train_log_df)


<ipython-input-51-db64f42192cc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predict'] = 0
<ipython-input-51-db64f42192cc>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for session_id in tqdm(session_ids):


/Users/imazekishouta/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## Testデータ: metaデータをもとに設定する

In [81]:
test_log_df['predict'] = 0

<ipython-input-81-90e3d148995f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_log_df['predict'] = 0


## TrainデータとTestデータに分けたログを合体させる

In [86]:
log_df2 = pd.concat([train_log_df2, test_log_df], axis=0)
assert len(log_df) == len(log_df2)

In [87]:
log_df2.to_csv('../output/cart_log.csv', index=None)

# 正解データの作成

In [41]:
is_predict = True

log_df2 = pd.read_csv('../output/cart_log.csv')
log_df2.head()

,session_id,register_number,date,hour,spend_time,display_action_id,user_id,value_1,name_1,kind_1,number_1,value_2,name_2,kind_2,number_2,unit_price,n_items,is_payment,predict
0,0,2003,2019-02-14,4,0,136,bE94Gct4vGiKM7z2Y79cjk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,0,2003,2019-02-14,4,32,209,bE94Gct4vGiKM7z2Y79cjk,4522646719765,おかかおにぎり ５９円,商品,1.0,NaN,NaN,NaN,NaN,59.0,1,0,0
2,0,2003,2019-02-14,4,37,209,bE94Gct4vGiKM7z2Y79cjk,4522646695663,梅（種なし）おにぎり５９,商品,1.0,NaN,NaN,NaN,NaN,59.0,1,0,0
3,0,2003,2019-02-14,4,52,209,bE94Gct4vGiKM7z2Y79cjk,4522646716641,紅鮭おにぎり７９円,商品,1.0,NaN,NaN,NaN,NaN,79.0,1,0,0
4,0,2003,2019-02-14,4,59,209,bE94Gct4vGiKM7z2Y79cjk,4522646695670,明太子おにぎり ７９円,商品,1.0,NaN,NaN,NaN,NaN,79.0,1,0,0


## 購入イベントのあったセッションのみに絞る

In [42]:
def get_paid_event(df):
    print('all sessions: ', len(df.session_id.unique()))

    paid_event_ids = list(df[df['is_payment']==1]['session_id'].unique())
    print('num sessions paid: ', len(paid_event_ids))

    return df[df['session_id'].isin(paid_event_ids)]

paid_event_df = get_paid_event(log_df2)

all sessions:  720175
num sessions paid:  618462


## 商品レコードのみに絞る

In [43]:
def get_item_record(df, cols=['session_id', 'value_1', 'n_items', 'predict']):
    return df[df['kind_1']=='商品'][cols]

paid_item_event_df = get_item_record(paid_event_df)
paid_item_event_df.head()

,session_id,value_1,n_items,predict
1,0,4522646719765,1,0
2,0,4522646695663,1,0
3,0,4522646716641,1,0
4,0,4522646695670,1,0
5,0,4522646720440,1,1


## 商品マスタと紐づけてcategory_idの取得

In [44]:
def join_category_id(df, product_master_df):
    df['value_1'] = df['value_1'].astype('int64')
    df = pd.merge(df, product_master_df[['JAN', 'category_id']], left_on='value_1', right_on='JAN', how='left')
    return df.drop(['value_1', 'JAN'], axis=1)

category_event_df = join_category_id(paid_item_event_df, product_master_df)
category_event_df.head()

,session_id,n_items,predict,category_id
0,0,1,0,617.0
1,0,1,0,617.0
2,0,1,0,617.0
3,0,1,0,617.0
4,0,1,1,617.0


## 対象商品のみに絞る

In [45]:
def get_target_category_record(df):
    target_category = [38, 110, 113, 114, 134, 171, 172, 173, 376, 435, 467, 537, 539, 629, 768]
    return df[df['category_id'].isin(target_category)]

target_category_event_df = get_target_category_record(category_event_df)
target_category_event_df.head()

,session_id,n_items,predict,category_id
35,7,1,0,768.0
36,7,1,0,768.0
38,7,1,0,768.0
39,7,1,0,768.0
40,7,1,0,134.0


## 予測対象の行のみに絞る

In [46]:
def get_target_category_record(df, is_predict=True):
    if is_predict:
        df = df[df['predict']==1]
    return df.drop(['predict'], axis=1)

target_row_event_df = get_target_category_record(target_category_event_df, is_predict=is_predict)
target_row_event_df.head()

,session_id,n_items,category_id
83,10,1,768.0
84,10,1,768.0
85,10,1,114.0
156,21,1,171.0
246,32,1,114.0


## 購入したか集計

In [47]:
def calc_is_payment(df):
    df = df.groupby(['session_id', 'category_id'])[['n_items']].sum().reset_index()
    df.loc[df['n_items'] > 0, 'n_items'] = 1
    df.loc[df['n_items'] < 0, 'n_items'] = 0
    return df
target_item_sum_df = calc_is_payment(target_row_event_df)
target_item_sum_df.head()

,session_id,category_id,n_items
0,10,114.0,1
1,10,768.0,1
2,21,171.0,1
3,32,114.0,1
4,38,629.0,1


## Pivot

In [48]:
def pivot_df(df):
    df = df.pivot(index="session_id", columns="category_id", values="n_items").reset_index()
    return df

train_payment_Y = pivot_df(target_item_sum_df)
train_payment_Y.head()

category_id,session_id,38.0,110.0,113.0,114.0,134.0,171.0,172.0,173.0,376.0,435.0,467.0,537.0,539.0,629.0,768.0
0,10,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,21,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,50,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Join meta.csv

In [49]:
def join_meta_df(df, meta_df):
    session_df = meta_df[['session_id']]
    df = pd.merge(session_df, df, on='session_id', how='left')
    df.fillna(0, inplace=True)
    return df

all_sessions_Y = join_meta_df(train_payment_Y, meta_df)
all_sessions_Y.head()

,session_id,38.0,110.0,113.0,114.0,134.0,171.0,172.0,173.0,376.0,435.0,467.0,537.0,539.0,629.0,768.0
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## カラム名を文字列方に

In [50]:
def col_name_to_str(df):
    cols = list(df.columns)
    str_cols = []
    for col in cols:
        if col == 'session_id':
            str_cols.append(col)
        else:
            str_cols.append(str(int(col)))
    df.columns = str_cols
    return df

all_sessions_Y = col_name_to_str(all_sessions_Y)
all_sessions_Y.head()

,session_id,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
all_sessions_Y[all_sessions_Y.session_id==7]

,session_id,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
7,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Train Test Split

In [52]:
def train_test_split(df, test_df):
    # test_Yいらない
    test_sessions = list(test_df['session_id'])
    test_Y = df[df['session_id'].isin(test_sessions)]
    train_Y = df[~df['session_id'].isin(test_sessions)]
    assert len(df) == len(test_Y) + len(train_Y)
    return train_Y, test_Y

train_Y, test_Y = train_test_split(all_sessions_Y, test_df)
display(train_Y.head())
display(test_Y.head())

,session_id,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,session_id,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
663721,663721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
663725,663725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
663737,663737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
663745,663745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
663747,663747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
if is_predict:
    print('is_predict')
    train_Y.to_csv('../output/train_Y.csv', index=None)
    test_Y.to_csv('../output/test_Y.csv', index=None)
else:
    train_Y.to_csv('../output/all_train_Y.csv', index=None)
    test_Y.to_csv('../output/test_Y.csv', index=None)

is_predict
